# MLOps Zoomcamp Homework 1

The goal of this homework is to train a simple model for predicting the duration of a ride.

- Module 1 Introduction  link: https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/01-intro

In [1]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [2]:
df_01 = pd.read_parquet("./data/yellow_tripdata_2023-01.parquet")
df_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [3]:
df_01.shape

(3066766, 19)

In [4]:
len(df_01.columns)

19

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [5]:
df_01['duration'] = df_01['tpep_dropoff_datetime'] - df_01['tpep_pickup_datetime']
df_01['duration'] = df_01['duration'].apply(lambda td: td.total_seconds() / 60)

In [6]:
df_01['duration'].std()

42.59435124195458

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [7]:
df_01_filtered = df_01[(df_01.duration >= 1) & (df_01.duration <= 60)]
df_01_filtered.shape

(3009173, 20)

In [8]:
filtered = df_01_filtered.shape[0]
total = df_01.shape[0]

print((filtered / total) * 100)

98.1220282212598


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [9]:
categorical = ['PULocationID', 'DOLocationID']

In [29]:
df_01_filtered['PULocationID'].nunique()

255

In [30]:
df_01_filtered['DOLocationID'].nunique()

260

In [12]:
df_01_filtered[categorical] = df_01_filtered[categorical].astype(str)

/tmp/ipykernel_171378/3069845389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_01_filtered[categorical] = df_01_filtered[categorical].astype(str)


In [13]:
train_dicts = df_01_filtered[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [14]:
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [15]:
len(dv.feature_names_)

515

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [16]:
target = 'duration'
y_train = df_01_filtered[target].values

In [17]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.649261931416412

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [18]:
df_02 = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')
df_02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2913955 entries, 0 to 2913954
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [19]:
df_02['duration'] = df_02['tpep_dropoff_datetime'] - df_02['tpep_pickup_datetime']
df_02['duration'] = df_02['duration'].apply(lambda td: td.total_seconds() / 60)

In [20]:
df_02['duration'].std()

42.84210176105113

In [21]:
df_02 = df_02[(df_02.duration >= 1) & (df_02.duration <= 60)]
df_02.shape

(2855951, 20)

In [23]:
df_02[categorical] = df_02[categorical].astype(str)

val_dicts = df_02[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_val = df_02[target].values

In [25]:
y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.8118162035401735